In [24]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

# Параметры
j = 5
alpha_j = 0.1 + 0.05 * j
n = 10
h = 1 / n

# Функция f(x)
def f(x):
    return alpha_j * np.exp(x) + (1 - alpha_j) * np.sin(x)

# Узлы интерполяции
x_vals = np.array([alpha_j + i * h for i in range(n + 1)])
f_vals = f(x_vals)

# Проверочные точки
x_star = x_vals[0] + (2 / 3) * h
x_star2 = x_vals[n // 2] + (1 / 2) * h
x_star3 = x_vals[-1] - (1 / 3) * h

# Производные f(x)
x = sp.Symbol('x')
f_sym = alpha_j * sp.exp(x) + (1 - alpha_j) * sp.sin(x)

# Построение сплайна
def calc_spline_coeffs():
    alpha = f_vals.copy()
    beta = [0] * (n + 1)
    gamma = [0] * (n + 1)
    delta = [0] * (n + 1)
    a_coef = [0] * n
    b_coef = [0] * n

    for i in range(1, n):
        temp1 = 6 * ((f_vals[i + 1] - f_vals[i]) - (f_vals[i] - f_vals[i - 1])) / h
        temp2 = 4 * h
        a_coef[i] = -h / temp2
        b_coef[i] = (temp1 - h * b_coef[i - 1]) / temp2

    for i in range(n - 1, 0, -1):
        gamma[i] = a_coef[i] * gamma[i + 1] + b_coef[i]

    f_derivative_2 = sp.diff(f_sym, x, 2)
    gamma[0] = f_derivative_2.subs(x, x_vals[0])
    gamma[n] = f_derivative_2.subs(x, x_vals[-1])

    for i in range(n, 0, -1):
        delta[i] = (gamma[i] - gamma[i - 1]) / h
        beta[i] = (f_vals[i] - f_vals[i - 1]) / h + (2 * gamma[i] + gamma[i - 1]) * h / 6

    return alpha, beta, gamma, delta

# Вычисление значения сплайна в точке x
def calc_f(x, x_i, a, b, c, d):
    dx = x - x_i
    return a + b * dx + 0.5 * c * dx ** 2 + (1 / 6) * d * dx ** 3

def calc_spline(x, nodes, P, n):
    for i in range(1, n):
        if nodes[i - 1] <= x <= nodes[i]:
            return calc_f(x, nodes[i], *P[i])
    return None


alpha, beta, gamma, delta = calc_spline_coeffs()

# Заполнение таблицы P из коэффициентов
P = [[0.0] * 4 for _ in range(n + 1)]
for i in range(n + 1):
    P[i][0] = alpha[i]  # значение функции в узле x_i
    P[i][1] = beta[i]   # первая производная в x_i
    P[i][2] = gamma[i]  # вторая производная в x_i
    P[i][3] = delta[i]  # третья производная в x_i

S_x_star = calc_spline(x_star, x_vals, P, n + 1)
S_x_star2 = calc_spline(x_star2, x_vals, P, n + 1)
S_x_star3 = calc_spline(x_star3, x_vals, P, n + 1)

df = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star, x_star2, x_star3],
    "f(x)": [f(x_star), f(x_star2), f(x_star3)],
    "S(x) (сплайн)": [S_x_star, S_x_star2, S_x_star3]
})

# Оценка погрешности
f_derivative_4 = sp.diff(f_sym, x, 4)
f_derivative_4_abs = sp.lambdify(x, sp.Abs(f_derivative_4), 'numpy')

a, b = x_vals[0], x_vals[-1]
x_test = np.linspace(a, b, 1000)
M_max = np.max(f_derivative_4_abs(x_test))
error_bound = h ** 4 * M_max

# Истинная ошибка
r_x_star = round(abs(f(x_star) - S_x_star), 6)
r_x_star2 = round(abs(f(x_star2) - S_x_star2), 6)
r_x_star3 = round(abs(f(x_star3) - S_x_star3), 6)

is_error_bound_valid = [
    r_x_star <= error_bound,
    r_x_star2 <= error_bound,
    r_x_star3 <= error_bound
]

error_table = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star, x_star2, x_star3],
    "r(ист)": [r_x_star, r_x_star2, r_x_star3],
    "Оценка погрешности (h^4 * M)": [error_bound] * 3,
    "M = max|f^(4)(x)|": [M_max] * 3,
    "Неравенство выполняется?": is_error_bound_valid
})

# Отображение таблиц
display(df)
display(error_table)

,Точка,Значение x,f(x),S(x) (сплайн)
0,x*,0.416667,0.793978,0.793930745224741
1,x**,0.900000,1.370024,1.370042
2,x***,1.316667,1.934961,1.93488681672318


,Точка,Значение x,r(ист),Оценка погрешности (h^4 * M),M = max|f^(4)(x)|,Неравенство выполняется?
0,x*,0.416667,4.8e-5,0.000198,1.984319,True
1,x**,0.900000,0.000018,0.000198,1.984319,True
2,x***,1.316667,7.4e-5,0.000198,1.984319,True
